In [1]:
import datetime as dt
import numpy as np 
import pandas as pd
from pandas.core import construction 

from scipy.stats import norm
from scipy.optimize import fsolve

In [2]:
class Merton_KMV():
    def __init__(self,D,E,t,r,sigma_e,x):
        self.D = D 
        self.E = E
        self.t = t
        self.r = r 
        self.sigma_e = sigma_e
        self.x = x
    def d1(self,A,D,sigma_a,r,t):
        self.d1_v = ( np.log(A/D) + ( r + sigma_a**2 /2 ) * t ) / ( sigma_a*np.sqrt(t) )
        return self.d1_v 
    def d2(self,A,D,sigma_a,r,t):
        self.d2_v = self.d1_v - sigma_a*np.sqrt(t)
        return self.d2_v 
    def func(self,x):
        sigma_a = x[0]
        A =  x[1] 
        d1_value = self.d1(A=A, D=self.D , sigma_a=sigma_a , r=self.r , t=self.t)
        d2_value = self.d2(A=A, D=self.D , sigma_a=sigma_a , r=self.r , t=self.t)
        
        return [ (A*norm.cdf(d1_value) - np.exp(-self.r*self.t)*self.D*norm.cdf(d2_value) - self.E) , ((A/self.E) * norm.cdf(d1_value) * sigma_a - self.sigma_e )]
    def fsolve(self):
        root = fsolve( self.func, x0=self.x)
        return root


In [3]:
# df = pd.read_csv(r'/Users/chen-lichiang/Desktop/data_20020701.csv')
df = pd.read_csv(r'/Users/chen-lichiang/Desktop/data_20011001.csv')


# df = pd.read_excel(r'/Users/chen-lichiang/Desktop/data_bonus.xls')

In [4]:
df['RET'] = np.log( df['RET'].values + 1)
df

,PERMNO,DLC,DLTT,DATE,COMNAM,RET,me,f,ym,TCMNOM_Y1,ir
0,11042,5015,13878,2000/10/2,WORLDCOM INC GA NEW,-0.031351,84678.05794,11954,200010,6.01,0.0601
1,11042,5015,13878,2000/10/3,WORLDCOM INC GA NEW,0.014753,85936.54288,11954,200010,6.01,0.0601
2,11042,5015,13878,2000/10/4,WORLDCOM INC GA NEW,-0.058143,81082.38669,11954,200010,6.01,0.0601
3,11042,5015,13878,2000/10/5,WORLDCOM INC GA NEW,-0.083189,74610.17844,11954,200010,6.01,0.0601
4,11042,5015,13878,2000/10/6,WORLDCOM INC GA NEW,-0.029342,72452.77569,11954,200010,6.01,0.0601
...,...,...,...,...,...,...,...,...,...,...,...
242,11042,7200,12494,2001/9/24,WORLDCOM INC GA NEW,0.109311,40836.58554,13447,200109,2.82,0.0282
243,11042,7200,12494,2001/9/25,WORLDCOM INC GA NEW,0.036264,42344.66870,13447,200109,2.82,0.0282
244,11042,7200,12494,2001/9/26,WORLDCOM INC GA NEW,0.030941,43675.33445,13447,200109,2.82,0.0282
245,11042,7200,12494,2001/9/27,WORLDCOM INC GA NEW,-0.019830,42817.79305,13447,200109,2.82,0.0282


In [5]:
def KMV_df(df):
    n=df.shape[0]
    df['RET'] = np.log( df['RET'].values + 1)
    sigma_e = []
    df['KMV debt'] = df['DLC'] + 0.5 * df['DLTT']
    for i in range(df.shape[0]+1,n,-1):
        e_values = np.std( df['RET'][-(n-i):i].values ) * np.sqrt(n)
        sigma_e.append(e_values)

    df = df[n-1:]
    df['sigma_e'] = list(reversed(sigma_e))  

    df = df [['DATE','RET','me','ir','KMV debt','sigma_e']]
    df = df.reset_index(drop=True)
    return df 

kmv_df = KMV_df(df=df)
kmv_df 

<ipython-input-5-51c601605b74>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sigma_e'] = list(reversed(sigma_e))


,DATE,RET,me,ir,KMV debt,sigma_e
0,2001/9/28,0.037243,44605.46645,0.0282,13447.0,0.739157


In [6]:
def d2(A,D,sigma_a,r,t):
    d2_v =  ( np.log(A/D) + (r-0.5*sigma_a**2) * t ) / ( sigma_a*np.sqrt(t) )
    return d2_v

def d1(A,D,sigma_a,r,t):
    d1_v =  ( np.log(A/D) + (r+0.5*sigma_a**2) * t ) / ( sigma_a*np.sqrt(t) )
    return d1_v

sigma_list = []
A_list = []
EDP = []
d2_list = []
d1_list = []
for i in range(kmv_df.shape[0]):
    t = 1
    d = kmv_df['KMV debt'][i]
    e = kmv_df['me'][i]
    r = kmv_df['ir'][i]
    sigma_e = kmv_df['sigma_e'][i]

    model = Merton_KMV(D=d,E=e,t=t,r=r,sigma_e=sigma_e,x=[ sigma_e , e+d ])
    ans = model.fsolve()
    
    sigma_list.append(ans[0])
    A_list.append(ans[1])

    
    d2_vlaue = d2(A=ans[1],D=d, sigma_a=ans[0],r=r,t=t)
    d2_list.append(d2_vlaue)
    d1_vlaue = d1(A=ans[1],D=d, sigma_a=ans[0] ,r=r,t=t)
    d1_list.append(d1_vlaue)
    EDP .append( norm.cdf(-1*d2_vlaue) )
    


kmv_df['sigma_a'] = sigma_list
kmv_df['A'] = A_list
kmv_df['d1'] = d1_list
kmv_df['d2'] = d2_list
kmv_df['EDP'] = EDP


kmv_df

,DATE,RET,me,ir,KMV debt,sigma_e,sigma_a,A,d1,d2,EDP
0,2001/9/28,0.037243,44605.46645,0.0282,13447.0,0.739157,0.573005,57655.569793,2.876237,2.303232,0.010633
